In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [22]:
lyrics_df = pd.read_csv('taylor_swift_lyrics.csv', encoding= "ISO-8859-1")
lyrics_df[lyrics_df['track_title'] == "Out of the Woods"].loc[:,'lyric']

3095                                    Looking at it now
3096                               It all seems so simple
3097                          We were lying on your couch
3098                                           I remember
3099                            You took a Polaroid of us
3100                    Then discovered (then discovered)
3101            The rest of the world was black and white
3102                       But we were in screaming color
3103                             And I remember thinking

3104    Are we out of the woods yet? Are we out of the...
3105    Are we out of the woods yet? Are we out of the...
3106    Are we in the clear yet? Are we in the clear yet?
3107      Are we in the clear yet? In the clear yet? Good
3108    Are we out of the woods yet? Are we out of the...
3109    Are we out of the woods yet? Are we out of the...
3110    Are we in the clear yet? Are we in the clear yet?
3111      Are we in the clear yet? In the clear yet? Good
3112          